In [91]:
from flask import Flask, request, jsonify, render_template
import os

In [92]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from sklearn.preprocessing import LabelEncoder
from spacy.lang.en import English

In [ ]:
loaded_model = tf.keras.models.load_model("skimlit_tribrid_model/")

In [207]:
app = Flask(__name__, template_folder='templates')

In [208]:
@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')

In [209]:
def split_chars(text):
    return " ".join(list(text))

def predictor(data):
    label_encoder = LabelEncoder()
    labels = ['OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS','BACKGROUND']
    labels = label_encoder.fit_transform(labels)
    abstract1 = data
    nlp = English() 
    sentencizer = nlp.create_pipe("sentencizer") 
    nlp.add_pipe('sentencizer') 
    doc = nlp(abstract1) 
    abstract_lines = [str(sent) for sent in list(doc.sents)]
    total_lines_in_sample = len(abstract_lines)
    sample_lines = []
    for i, line in enumerate(abstract_lines):
        sample_dict = {}
        sample_dict["text"] = str(line)
        sample_dict["line_number"] = i
        sample_dict["total_lines"] = total_lines_in_sample - 1
        sample_lines.append(sample_dict)
    test_abstract_line_numbers = [line["line_number"] for line in sample_lines]
    test_abstract_line_numbers_one_hot = tf.one_hot(test_abstract_line_numbers, depth=15) 
    test_abstract_total_lines = [line["total_lines"] for line in sample_lines]
    test_abstract_total_lines_one_hot = tf.one_hot(test_abstract_total_lines, depth=20)
    abstract_chars = [split_chars(sentence) for sentence in abstract_lines]
    test_abstract_pred_probs = loaded_model.predict(x=(test_abstract_line_numbers_one_hot,
                                                   test_abstract_total_lines_one_hot,
                                                   tf.constant(abstract_lines),
                                                   tf.constant(abstract_chars)))
    test_abstract_preds = tf.argmax(test_abstract_pred_probs, axis=1)
    test_abstract_pred_classes = [label_encoder.classes_[i] for i in test_abstract_preds]
    o = ''
    m = ''
    r = ''
    c = ''
    b = ''
    for i, line in enumerate(abstract_lines):
        if(test_abstract_pred_classes[i]) == 'OBJECTIVE':
            o = o + line + ' '
        elif(test_abstract_pred_classes[i]) == 'METHODS':
            m = m + line + ' '
        elif(test_abstract_pred_classes[i]) == 'RESULTS':
            r = r + line + ' '
        elif(test_abstract_pred_classes[i]) == 'CONCLUSIONS':
            c = c + line + ' '
        elif(test_abstract_pred_classes[i]) == 'BACKGROUND':
            b = b + line + ' '
    if(len(o)) > 0:
        o = 'OBJECTIVE: '+ o
    if(len(m)) > 0:
        m='METHODS: '+m
    if(len(r)) > 0:
        r='RESULTS: '+r
    if(len(c)) > 0:
        c='CONCLUSIONS: '+c
    if(len(b)) > 0:
        b='BACKGROUND: '+b 
    return o

In [210]:
@app.route('/result', methods = ['POST', 'GET'])
def result():
    string2 = request.form["paragraph_text"]
    prediction = predictor(string2)
    return render_template('result.html',prediction = prediction)   

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Nov/2021 18:26:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2021 18:26:05] "GET /templates/style.css HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2021 18:26:09] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2021 18:26:09] "GET /templates/style.css HTTP/1.1" 404 -


In [ ]:
This RCT examined the efficacy of a manualized social intervention for children with HFASDs. Participants were randomly assigned to treatment or wait-list conditions. Treatment included instruction and therapeutic activities targeting social skills, face-emotion recognition, interest expansion, and interpretation of non-literal language. A response-cost program was applied to reduce problem behaviors and foster skills acquisition. Significant treatment effects were found for five of seven primary outcome measures (parent ratings and direct child measures). Secondary measures based on staff ratings (treatment group only) corroborated gains reported by parents. High levels of parent, child and staff satisfaction were reported, along with high levels of treatment fidelity. Standardized effect size estimates were primarily in the medium and large ranges and favored the treatment group.